In [69]:
import tensorflow as tf

# load the model
model = tf.keras.models.load_model('unet_model')

In [70]:
import os

data_dir = 'data'
mask_dir = "masks"
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
test_masks_dir = os.path.join(mask_dir, 'test')
train_masks_dir = os.path.join(mask_dir, 'train')


In [71]:
# Run on all images in the test set and save the true and predicted masks, and write the filenames to a text file
# Create word document with images and predicted contours
import cv2
import numpy as np
import os
import tensorflow as tf
from docx import Document
from docx.shared import Inches

data_dir = 'data'
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')

# Load all images to numpy array to apply model
test_images = []
for filename in os.listdir(test_dir):
    if filename.endswith('.png'):
        test_images.append(os.path.join(test_dir, filename))

for filename in os.listdir(train_dir):
    if filename.endswith('.png'):
        test_images.append(os.path.join(train_dir, filename))

# Load model
model = tf.keras.models.load_model('internal_mask_unet_model')

# Create word document
document = Document()
document.add_heading('Predicted Internal Contours', 0)

# Create table with the images, expected masks and predicted contours
table = document.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Image'
hdr_cells[1].text = 'Expected Mask'
hdr_cells[2].text = 'Predicted Mask'

# Loop through all images and apply model
for directory in [test_dir]:
    for file in os.listdir(directory):

        img_path = os.path.join(directory, file)
        # Load image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        img = img / 255.0
        img = np.reshape(img, (1, 128, 128, 3))

        # Predict mask
        mask = model.predict(img)
        mask = np.reshape(mask, (1, 128, 128, 2))
        # mask = mask * 255.0
        # mask = mask.astype(np.uint8)
        mask = tf.argmax(mask, axis=-1)
        mask = mask[..., tf.newaxis]
        # Reshape mask to 128x128
        mask = np.reshape(mask, (128, 128))
        mask = mask * 255.0
        mask = mask.astype(np.uint8)
        os.makedirs('predicted_internal_contours', exist_ok=True)
        cv2.imwrite('predicted_internal_contours\\' + file, mask)

        # Load expected mask
        # remove data from path
        tmp_directory = directory.replace('data\\', '')
        mask_path = os.path.join(mask_dir, tmp_directory.split('/')[-1], file)
        expected_mask = cv2.imread(mask_path)
        expected_mask = cv2.resize(expected_mask, (128, 128))
        expected_mask = expected_mask[:, :, 0]
        expected_mask = expected_mask.astype(np.uint8)

        # Add image, expected mask and predicted mask to table
        row_cells = table.add_row().cells
         # add file name to each row
        row_cells[0].text = file
        row_cells[0].add_paragraph().add_run().add_picture(img_path, width=Inches(1.0))
        row_cells[1].add_paragraph().add_run().add_picture(mask_path, width=Inches(1.0))
        row_cells[2].add_paragraph().add_run().add_picture('predicted_internal_contours/' + file, width=Inches(1.0))

# Save word document
document.save('predicted_internal_contours.docx')

1/1 [==============================] - 0s 62ms/step
